In [1]:
import os
import pandas as pd

In [2]:
remove = pd.read_csv(
    os.path.join(
        os.getcwd().replace("ADNI2-T1-AD-CN", "ADNI1-T1-AD-CN"), r"ADNI1-Collect.csv"
    )
)

In [3]:
sub_adni1 = list(remove["Subject ID"].unique())

In [4]:
df_id = pd.read_csv(os.path.join(os.getcwd(), r"ADNI2-Total.csv"))

df_id_not_type = df_id[~df_id["Imaging Protocol"].str.contains("Type")]
df_id_not_type["Imaging Protocol"] = (
    df_id_not_type["Imaging Protocol"] + ";Acquisition Type=3D"
)
df_id_type = df_id[df_id["Imaging Protocol"].str.contains("Type")]
df_id = pd.concat([df_id_not_type, df_id_type])


df_id[["Manufacturer", "Model", "Type"]] = df_id["Imaging Protocol"].str.extract(
    "Manufacturer=(.+);Mfg Model=(.+);Acquisition Type=(.+)", expand=True
)
df_id["Type"] = df_id["Type"].fillna("3D")
df_id = df_id[df_id["Type"] == "3D"]

df_id = df_id.replace(
    {
        "GE MEDICAL SYSTEMS": "GE",
        "Philips Healthcare": "Philips",
        "Philips Medical Systems": "Philips",
        "SIEMENS|PixelMed": "Siemens",
        "SIEMENS": "Siemens",
    },
    regex=False,
)

df_id = df_id[~df_id["Subject ID"].isin(sub_adni1)]

# df_id = df_id[df_id["Description"].str.contains("MP")]

C:\Users\doxda\AppData\Local\Temp\ipykernel_13236\3247019251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_not_type["Imaging Protocol"] = (


In [5]:
df_id.head()

,Subject ID,Sex,Research Group,Visit,Age,Description,Imaging Protocol,Image ID,Manufacturer,Model,Type
57,002_S_4213,F,CN,ADNI2 Screening MRI-New Pt,78.1,MPRAGE SENSE2,Manufacturer=Philips Medical Systems;Mfg Model...,254579,Philips,Intera,3D
58,002_S_4213,F,CN,ADNI2 Screening MRI-New Pt,78.1,Field Mapping,Manufacturer=Philips Medical Systems;Mfg Model...,254580,Philips,Intera,3D
59,002_S_4213,F,CN,ADNI2 Screening MRI-New Pt,78.1,MPRAGE,Manufacturer=Philips Medical Systems;Mfg Model...,254582,Philips,Intera,3D
60,002_S_4213,F,CN,ADNI2 Month 3 MRI-New Pt,78.4,MPRAGE SENSE2,Manufacturer=Philips Medical Systems;Mfg Model...,273216,Philips,Intera,3D
61,002_S_4213,F,CN,ADNI2 Month 3 MRI-New Pt,78.4,Field Mapping,Manufacturer=Philips Medical Systems;Mfg Model...,273219,Philips,Intera,3D


In [6]:
df_id["Image ID"].count()

3543

In [7]:
firsts = df_id.groupby(["Subject ID", "Visit"], as_index=False).first()

In [8]:
firsts["Subject ID"].nunique()

381

In [9]:
firsts.Visit.unique()

array(['ADNI2 Month 3 MRI-New Pt', 'ADNI2 Month 6-New Pt',
       'ADNI2 Screening MRI-New Pt', 'ADNI2 Year 1 Visit',
       'ADNI2 Year 2 Visit', 'ADNI2 Year 4 Visit', 'ADNI2 Year 5 Visit',
       'ADNI2 Year 3 Visit', 'ADNI2 Tau-only visit'], dtype=object)

In [10]:
firsts = firsts[firsts["Visit"].isin(["ADNI2 Screening MRI-New Pt"])]

In [11]:
firsts["Image ID"].nunique()

381

In [12]:
firsts.groupby(["Research Group"])["Image ID"].nunique()

Research Group
AD    159
CN    222
Name: Image ID, dtype: int64

In [13]:
firsts.groupby(["Manufacturer", "Research Group"])["Image ID"].nunique()

Manufacturer  Research Group
GE            AD                 50
              CN                 64
Philips       AD                 34
              CN                 45
Siemens       AD                 75
              CN                113
Name: Image ID, dtype: int64

In [14]:
firsts["dot"] = 1

In [15]:
firsts[["Image ID", "dot"]].to_csv("ID.csv", index=False)